In [24]:
use strict;
use warnings;
use Data::Dump qw(dump);
use List::Util qw(zip sum);
use sml; # Statistical Machine Learning Library
use AI::MXNet qw(mx);

In [45]:
my $random_algorithm = sub {
    my ($self, $train, $test) = @_;

    if (ref($train) eq 'AI::MXNet::NDArray' and ref($test) eq 'AI::MXNet::NDArray') {
        
        my $train_labels = $train->slice_axis(axis => 1, begin => $train->shape->[1] - 1, end => $train->shape->[1]);

        my @labels = $train_labels->aspdl->list;

        my %seen;
        my @unique = grep { !$seen{$_}++ } @labels;

        my $num_test_rows = $test->shape->[0];
        my @rand_indices = map { int(rand(@unique)) } (1 .. $num_test_rows);
        my @predicted_values = map { $unique[$_] } @rand_indices;

        my $predictions = mx->nd->array(\@predicted_values);
        return $predictions;
    }
    else {

        my @output_values = map { $_->[-1] } @$train;
        my %seen;
        my @unique = grep { !$seen{$_}++ } @output_values;

        return [ map { $unique[int(rand(@unique))] } @$test ];
    }
};

sml->add_to_class('random_algorithm', \&$random_algorithm);

*sml::random_algorithm

Warning: Subroutine sml::random_algorithm redefined at /usr/local/share/perl5/5.34/x86_64-linux-thread-multi/sml.pm line 16.


In [47]:
mx->random->seed(1);

my $train = mx->nd->array([[0], [1], [0], [1], [0], [1], [1]]);
my $test  = mx->nd->array([[undef], [undef], [undef], [undef]]);

my $predictions = sml->random_algorithm($train, $test);
print $predictions->aspdl;  # Ejemplo esperado: 0 0 1 0

print $train ->at(-1)->aspdl;

[0 0 1 0][1]

1

In [52]:
my $zero_rule_algorithm_classification = sub {
    my ($self, $train, $test) = @_;

    if (ref($train) eq 'AI::MXNet::NDArray' and ref($test) eq 'AI::MXNet::NDArray') {

        my $label_col = $train->slice_axis(axis => 1, begin => $train->shape->[1] - 1, end => $train->shape->[1]);

        my @labels = $label_col->aspdl->list;

        my %counter;
        $counter{$_}++ for @labels;

        my $prediction_value = (sort { $counter{$b} <=> $counter{$a} } keys %counter)[0];

        my $num_test_rows = $test->shape->[0];
        my $predictions = mx->nd->full([$num_test_rows], $prediction_value);

        return $predictions;
    }
    else {

        my @output_values = map { $_->[-1] } @$train;
        my %counter = ();
        map { $counter{$_}++ } @output_values; # Counts
        my $prediction = (sort { $counter{$b} <=> $counter{$a} } keys %counter)[0]; # Find the most frequent
        return [(($prediction) x scalar(@$test))]; # predictions
    }
};

sml->add_to_class('zero_rule_algorithm_classification', \&$zero_rule_algorithm_classification);

*sml::zero_rule_algorithm_classification

Warning: Subroutine sml::zero_rule_algorithm_classification redefined at /usr/local/share/perl5/5.34/x86_64-linux-thread-multi/sml.pm line 16.


In [62]:
mx->random->seed(1);

my $train = mx->nd->array([[0], [0], [0], [1], [0], [1], [1]]);
my $test  = mx->nd->array([[undef], [undef], [undef], [undef]]);

my $predictions = sml->zero_rule_algorithm_classification($train, $test);
print $predictions->aspdl;  # Ejemplo esperado: 0 0 1 0


[0 0 0 0]

1

In [64]:
my $zero_rule_algorithm_regression = sub {
    my ($self, $train, $test) = @_;

    if (ref $train eq 'AI::MXNet::NDArray' && ref $test eq 'AI::MXNet::NDArray') {
        my $output_values_nd = $train->slice_axis(axis => 1, begin => $train->shape->[1] - 1, end => $train->shape->[1]);;
        my $sum_nd = $output_values_nd->sum();
        my $count = $output_values_nd->shape->[0];
        my $average_nd = $sum_nd / $count;
        my $prediction_value = $average_nd->asscalar();
        return mx->nd->full([$test->shape->[0]], $prediction_value);
    } elsif (ref $train eq 'ARRAY' && ref $test eq 'ARRAY') {
        my @output_values = map { $_->[-1] } @$train;
        my $sum_values = sum(@output_values);
        my $average = $sum_values / scalar(@output_values);
        my $prediction = sprintf '%.1f', $average;
        return [(($prediction) x scalar(@$test))];
    }
};
sml->add_to_class('zero_rule_algorithm_regression', \&$zero_rule_algorithm_regression);

*sml::zero_rule_algorithm_regression

In [69]:
my $train = mx->nd->array([[1, 2, 3],
 [4, 5, 6],
 [7, 8, 9]]);
my $test  = mx->nd->array([[undef], [undef], [undef], [undef]]);

my $predictions = sml->zero_rule_algorithm_regression($train, $test);
print $predictions->aspdl;  # Ejemplo esperado: 0 0 1 0

[6 6 6 6]

1